In [1]:
from numpy import array
import pandas as pd

In [2]:
"""
Taken from https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
"""
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30 40 50] 60
[20 30 40 50 60] 70
[30 40 50 60 70] 80
[40 50 60 70 80] 90


In [3]:
prices = pd.read_csv('../Functions/csv_files/prices/train.csv')

In [4]:
apple = prices[prices['ticker'] == 'AAPL.US']

In [23]:
X, y = split_sequence(apple['close'].tolist(), 200)

In [20]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras

In [74]:
temp = X.reshape((len(X), 1, 200))
temp

array([[[ 28.7392,  27.2608,  25.2448, ...,  16.3744,  14.2464,
          14.3808]],

       [[ 27.2608,  25.2448,  25.872 , ...,  14.2464,  14.3808,
          15.12  ]],

       [[ 25.2448,  25.872 ,  26.6336, ...,  14.3808,  15.12  ,
          15.2544]],

       ...,

       [[125.57  , 124.76  , 120.53  , ..., 176.28  , 180.33  ,
         179.29  ]],

       [[124.76  , 120.53  , 119.99  , ..., 180.33  , 179.29  ,
         179.38  ]],

       [[120.53  , 119.99  , 123.39  , ..., 179.29  , 179.38  ,
         178.2   ]]])

In [91]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 200)))
model.add(Dense(1))
model.compile(optimizer='adam', loss = 'mse')

In [92]:
model.fit(
    temp, y, batch_size=64, epochs=50, verbose=1
)

Epoch 1/50
159/159 [==============================] - 4s 6ms/step - loss: 1582.1686
Epoch 2/50
159/159 [==============================] - 1s 7ms/step - loss: 360.5143
Epoch 3/50
159/159 [==============================] - 1s 7ms/step - loss: 309.4944
Epoch 4/50
159/159 [==============================] - 1s 6ms/step - loss: 432.4383
Epoch 5/50
159/159 [==============================] - 1s 4ms/step - loss: 236.5597
Epoch 6/50
159/159 [==============================] - 1s 7ms/step - loss: 219.0947
Epoch 7/50
159/159 [==============================] - 1s 5ms/step - loss: 202.3853
Epoch 8/50
159/159 [==============================] - 1s 5ms/step - loss: 243.0593
Epoch 9/50
159/159 [==============================] - 1s 6ms/step - loss: 188.0212
Epoch 10/50
159/159 [==============================] - 1s 8ms/step - loss: 230.0663
Epoch 11/50
159/159 [==============================] - 1s 6ms/step - loss: 194.0229
Epoch 12/50
159/159 [==============================] - 1s 6ms/step - loss: 156.8831


In [94]:
sample = temp[0].reshape(1, 1, 200)
print(model.predict(sample))
print(y[0])

1/1 [==============================] - 0s 53ms/step
[[15.079105]]
15.12


In [96]:
dev_prices = pd.read_csv('../Functions/csv_files/prices/dev.csv')

In [99]:
dev_x, dev_y = split_sequence(dev_prices['close'].tolist(), 200)

In [106]:
sample_dev = dev_x[:100]

In [108]:
sample_dev = sample_dev.reshape(len(sample_dev), 1, 200)

In [110]:
model.predict(sample_dev)

4/4 [==============================] - 0s 5ms/step


array([[15.701584],
       [15.381823],
       [15.282722],
       [15.492168],
       [15.823944],
       [15.846463],
       [15.569901],
       [15.902837],
       [15.941616],
       [16.648014],
       [17.416222],
       [17.500462],
       [17.519484],
       [17.323837],
       [17.083954],
       [17.128708],
       [16.9255  ],
       [17.088137],
       [17.043396],
       [17.362114],
       [17.105736],
       [16.908699],
       [16.728142],
       [16.740974],
       [16.656622],
       [16.616783],
       [16.929148],
       [17.073193],
       [16.438042],
       [16.748129],
       [16.56091 ],
       [16.958588],
       [16.591223],
       [16.913324],
       [16.663776],
       [16.455162],
       [16.065987],
       [16.228588],
       [16.68996 ],
       [16.983562],
       [17.14768 ],
       [17.25597 ],
       [17.096598],
       [17.324322],
       [17.233372],
       [17.933546],
       [18.210484],
       [17.905348],
       [17.987782],
       [18.382034],


In [113]:
sample_dev_y = dev_y[:100]

In [114]:
sample_dev_y

array([15.5   , 15.55  , 15.98  , 16.07  , 15.999 , 15.9   , 16.    ,
       17.22  , 17.4   , 17.42  , 17.2   , 16.85  , 16.8   , 16.55  ,
       17.    , 17.03  , 17.05  , 16.91  , 17.14  , 17.    , 16.58  ,
       16.81  , 16.89  , 16.9   , 16.79  , 16.85  , 16.95  , 16.82  ,
       16.95  , 16.85  , 17.05  , 16.57  , 16.89  , 17.05  , 17.039 ,
       17.    , 16.89  , 16.93  , 17.2   , 17.1   , 17.06  , 17.22  ,
       17.45  , 17.94  , 17.88  , 18.    , 17.99  , 18.03  , 18.93  ,
       19.1   , 19.4   , 19.26  , 19.5   , 20.01  , 21.04  , 20.02  ,
       20.56  , 21.0108, 21.09  , 21.0975, 21.15  , 21.32  , 21.32  ,
       21.25  , 21.29  , 21.36  , 20.93  , 20.25  , 20.23  , 20.63  ,
       20.3   , 19.99  , 20.71  , 19.88  , 19.82  , 19.55  , 19.4   ,
       19.45  , 19.38  , 19.29  , 19.28  , 19.    , 19.05  , 19.13  ,
       18.94  , 18.95  , 18.76  , 18.85  , 18.86  , 19.    , 19.    ,
       19.48  , 19.55  , 19.99  , 19.93  , 19.93  , 19.9   , 19.71  ,
       19.81  , 20. 